In [ ]:
import requests
import numpy as np
import json
import time

In [ ]:
# API key
key = "Deleted"
# URL for getting match by sequence
by_sequence = "http://api.steampowered.com/IDOTA2Match_570/GetMatchHistoryBySequenceNum/v1?matches_requested=100&start_at_match_seq_num="
# Starting sequence number
seq_num = 5273209735

In [ ]:
# 4 x n x n array
# First dimension index 0: wins with hero 1: games played with 2: wins agains 3: games played against
# Second Dimension Hero being checked
# Third Dimension Hero to compare winrate to
# There are 122 heroes, but hero_ids skip some numbers so highest ID id 136, therefore using max array index of 136
winrateArray = np.zeros((4, 137, 137))

In [ ]:
# waits for 100000 matches to be counted
while(winrateArray.sum()/150 < 100000):
    
    works = True
    
    # progress check as function runs
    print('matches used: ' + str(winrateArray.sum()/150) + ' seq num: ' + str(seq_num))
    
    # timer to not pass api call limit
    time.sleep(6)
    
    # catch in case api call limit is blocking or other possible errors
    try:
        r = requests.get(by_sequence + str(seq_num) + "&key=" + key).json()
    except:
        works = False
        print('caught error')
        seq_num += 1
        time.sleep(10)
    
    if(works):
        # runs through each match to find hero winrates
        for match in r['result']['matches']:
            noLeaver = True
            seq_num = match['match_seq_num'] + 1
            
            # ensures only ranked matches with no leaver and duration over 10 minutes are counted
            if(match['lobby_type'] == 7 and match['human_players'] == 10 and 
               match['duration'] > 600):
                for index in range(10):
                    if(match['players'][index]['leaver_status'] > 1):
                        noLeaver = False 
            else:
                noLeaver=False

            if(noLeaver):
        
                # adds matches with other heroes to array
                if(match['radiant_win']):
                    for hero in match['players']:
                        if(hero['player_slot'] < 20):
                            for others in match['players']:
                                if(others['player_slot'] < 20):
                                    winrateArray[0][hero['hero_id']][others['hero_id']] += 1
                                    winrateArray[1][hero['hero_id']][others['hero_id']] += 1
                                else:
                                    winrateArray[2][hero['hero_id']][others['hero_id']] += 1
                                    winrateArray[3][hero['hero_id']][others['hero_id']] += 1
                        else:
                            for others in match['players']:
                                if(others['player_slot'] > 20):
                                    winrateArray[0][hero['hero_id']][others['hero_id']] += 0
                                    winrateArray[1][hero['hero_id']][others['hero_id']] += 1
                                else:
                                    winrateArray[2][hero['hero_id']][others['hero_id']] += 0
                                    winrateArray[3][hero['hero_id']][others['hero_id']] += 1
                else:
                    for hero in match['players']:
                        if(hero['player_slot'] < 20):
                            for others in match['players']:
                                if(others['player_slot'] < 20):
                                    winrateArray[0][hero['hero_id']][others['hero_id']] += 0
                                    winrateArray[1][hero['hero_id']][others['hero_id']] += 1
                                else:
                                    winrateArray[2][hero['hero_id']][others['hero_id']] += 0
                                    winrateArray[3][hero['hero_id']][others['hero_id']] += 1
                        else:
                            for others in match['players']:
                                if(others['player_slot'] > 20):
                                    winrateArray[0][hero['hero_id']][others['hero_id']] += 1
                                    winrateArray[1][hero['hero_id']][others['hero_id']] += 1
                                else:
                                    winrateArray[2][hero['hero_id']][others['hero_id']] += 1
                                    winrateArray[3][hero['hero_id']][others['hero_id']] += 1

In [ ]:
# get official updated dota hero list
herolist = requests.get('http://api.steampowered.com/IEconDOTA2_570/GetHeroes/v1?key=' + key).json()['result']['heroes']

In [ ]:
# dict of hero winrates
winrates = {}

# calculate hero winrates with other heroes from array
for hero in herolist:
    
    # id of current hero
    heroID = hero['id']
    # name of current hero
    heroName = hero['name'].split('hero_')[-1]
    
    # dict to be filled with current hero's winrate with each other hero
    winrates_with = {}
    # dict to be filled with current hero's winrate against each other hero
    winrates_vs = {}
    
    # calculates current heroes winrate with and against each other hero to fill dicts
    for other in herolist:
        
        # name of hero to be compared to current
        name = other['name'].split('hero_')[-1]
        # id of hero to be compared to current
        otherID = other['id']
        
        # ensures at least one match played with both heroes on same team
        if(winrateArray[1][heroID][otherID] != 0):
            # winrate of both heroes on same team
            winrates_with[name] = winrateArray[0][heroID][otherID]/winrateArray[1][heroID][otherID]
        else:
            winrates_with[name] = None
        
        # ensures at least one match played with both heroes on opposing teams
        if(winrateArray[3][heroID][otherID] != 0):
            # winrate of current hero with compared hero on opposing team
            winrates_vs[name] = winrateArray[2][heroID][otherID]/winrateArray[3][heroID][otherID]
        else:
            winrates_vs[name] = None
    
    # appends current hero winrates as empty dict to be filled
    winrates[heroName] = {}
    
    # appends current hero winrate dicts with or against all other heroes
    winrates[heroName]['winrate_with'] = winrates_with
    winrates[heroName]['winrate_vs'] = winrates_vs

In [ ]:
# export data to external json file
with open('test.json', 'w') as test:
    json.dump(winrates, test)